# Getting Started with LouieAI Notebook Interface

This notebook demonstrates the basic usage of the LouieAI notebook-friendly API (`lui`).

## Authentication Requirements

LouieAI uses PyGraphistry for authentication. Before running this notebook, you need:

1. **PyGraphistry Account**: Sign up for free at [hub.graphistry.com](https://hub.graphistry.com)
2. **Set Credentials**: Use one of these methods:

   ```bash
   # Option 1: Environment variables (recommended for notebooks)
   export GRAPHISTRY_USERNAME="your_username"
   export GRAPHISTRY_PASSWORD="your_password"
   ```
   
   ```python
   # Option 2: Authenticate in code
   import graphistry
   graphistry.register(api=3, server="hub.graphistry.com", 
                      username="your_username", password="your_password")
   ```

## Setup

First, make sure you have LouieAI installed and your credentials configured.

In [ ]:
import os

import graphistry

from louieai import louieai  # Note: louieai not louie

# Authenticate with Graphistry
# Uses GRAPHISTRY_USERNAME and GRAPHISTRY_PASSWORD from environment
g = graphistry.register(
    api=3,
    server=os.environ.get("GRAPHISTRY_SERVER", "hub.graphistry.com"),
    username=os.environ.get("GRAPHISTRY_USERNAME"),
    password=os.environ.get("GRAPHISTRY_PASSWORD"),
)

# Configure Louie server URL
# Default to production server, or set LOUIE_SERVER env var for custom
# Examples:
#   export LOUIE_SERVER="http://localhost:8000"  # Local development
#   export LOUIE_SERVER="https://den.louie.ai"    # Production (default)
LOUIE_SERVER_URL = os.environ.get("LOUIE_SERVER", "https://den.louie.ai")

# Create the Louie interface
lui = louieai(g, server_url=LOUIE_SERVER_URL)

lui  # Display status

## Basic Queries

Making queries with `lui` is simple - just call it like a function:

In [2]:
# Ask a simple question
response = lui("What is the capital of France?")

# The response is automatically stored and accessible
print(f"Answer: {lui.text}")

Answer: The capital of France


## Working with Data

LouieAI can generate and analyze data. Let's create a sample dataset:

In [ ]:
# Request data generation with explicit "dataframe" mention
lui(
    "Create a dataframe with 10 products containing columns: "
    "product_name (string), category (string), price (float), units_sold (integer)"
)

# Access the generated dataframe
df = lui.df
if df is not None:
    print(f"Generated DataFrame with shape: {df.shape}")
    print("\nFirst few rows:")
    display(df.head())
else:
    print(
        "No dataframe was generated. Try asking for 'a table' "
        "or 'dataframe' explicitly."
    )

## Analyzing Data

Once you have data, you can ask questions about it:

In [ ]:
# Simple analysis
lui("What is the maximum price in the dataframe?")

# The text explanation is available
print(f"Answer: {lui.text}")

# If a dataframe is returned, display it
if lui.df is not None:
    print("\nReturned data:")
    display(lui.df)

### Need more options?

For advanced upload patterns including BytesIO, PIL Images, and more, see the [Data Upload Guide](02-data-upload.ipynb).

## Uploading and Analyzing Your Own Data

LouieAI supports uploading your own DataFrames, images, and documents for analysis:

In [5]:
import numpy as np
import pandas as pd

# Create your own DataFrame (or load from CSV/Excel)
my_data = pd.DataFrame(
    {
        "date": pd.date_range("2024-01-01", periods=30, freq="D"),
        "sales": np.random.randn(30).cumsum() + 100,
        "visitors": np.random.randint(50, 200, 30),
        "region": np.random.choice(["North", "South", "East", "West"], 30),
    }
)

print("Sample of your data:")
my_data.head()

Sample of your data:


,date,sales,visitors,region
0,2024-01-01,100.299330,168,East
1,2024-01-02,101.691448,94,East
2,2024-01-03,101.802578,110,West
3,2024-01-04,101.591971,129,South
4,2024-01-05,101.603414,193,East


In [6]:
# Analyze images
# lui("What's in this image?", "chart.png")
# lui("Extract text from this screenshot", "screenshot.jpg")

# Analyze documents
# lui("Summarize this document", "report.pdf")
# lui("Extract key points from this presentation", "slides.pptx")

# The AI will process your files and provide analysis
# print("Analysis:", lui.text)

### Loading Data from Files

You can also load and analyze data from CSV, Excel, or other formats:

In [7]:
# Load data from CSV
# df = pd.read_csv('sales_data.csv')
# lui("Find patterns in this sales data", df)

# Load from Excel
# df = pd.read_excel('report.xlsx', sheet_name='Sales')
# lui("What are the key insights?", df)

# The data is now uploaded and analyzed by LouieAI
# print("Analysis:", lui.text)

In [ ]:
# Upload and analyze your DataFrame with a simple question
lui("How many rows are in this sales data?", my_data)

# The AI analyzes YOUR data
print("Analysis:", lui.text)

# Access any returned visualizations or processed data
if lui.df is not None:
    print("\nProcessed results:")
    display(lui.df)

## Session History

The `lui` interface maintains a history of your interactions:

In [9]:
# Access previous responses by index
# lui[-1]  # Last response
# lui[-2]  # Second to last response

# Check the number of interactions
print(f"Total interactions: {len(lui._history)}")

Total interactions: 4


## Enabling Traces

For complex queries, you might want to see the AI's reasoning process:

In [ ]:
# Enable traces for the session
lui.traces = True

# Now queries will show reasoning steps
lui("What is 100 + 50?")

# Disable traces
lui.traces = False

# Or enable traces for just one query
lui("What is the sum of 1 to 10?", traces=True)

## Error Handling

The interface handles errors gracefully:

In [11]:
# Check if the last operation had errors
if lui.has_errors:
    print("Last operation had errors:", lui.errors)
else:
    print("No errors in last operation")

Last operation had errors: [{'id': 'B_eZ6avA8Q', 'type': 'ExceptionElement', 'parent': 'B_fIgjbHsW', 'text': 'ERROR: ToolAdapter:_run:func:Exception: Failed to load requirements: HTTPConnectionPool(host=\'maestro-caddy\', port=80): Max retries exceeded with url: /maestro/reqs (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7f41334e2480>: Failed to resolve \'maestro-caddy\' ([Errno -3] Temporary failure in name resolution)"))', 'traceback': None}]


## Tips and Tricks

Here are some helpful patterns for using the notebook interface:

1. **Use clear prompts**: Be specific about what you want to analyze
2. **Check outputs**: Always verify `lui.df` and `lui.text` after queries
3. **Enable traces for debugging**: Use `traces=True` when troubleshooting
4. **Leverage history**: Access previous results with `lui[-1]`, `lui[-2]`, etc.
5. **Upload various formats**: DataFrames, images, PDFs all work seamlessly